## Instances initialization

In [5]:
from main import *
import random
import math

items, knapsack, optimum = open_instance("../instances/chubeas/OR5x100/OR5x100.dat")
#ks = Knapsack(knapsack[0], items[0])
#print(ks.fit())

## GWO
### Population initialization

In [10]:
# Accumulated resources as list of dimensions ?
def pop_init(population_size, list_items, list_constraints):
    """
    Initialize a population of "elite" solutions

    :param population_size: Size of the population
    :type population_size: int
    :param list_items: List of items for the MKP
    :type list_items: list of Item
    :param list_constraints: List of constraints for this knapsack
    :type list_constraints: list of int

    :rtype: list of Knapsack
    :return: The population
    """
    population = []

    for k in range(population_size):
        accumulated_resources = [0]*len(list_constraints)
        ks = Knapsack(list_constraints, list_items)
        for j in range(len(list_items)):
            # In descending of the pseudo utility -> sort before

            # Test if all accumulated resources are inferior to the constraints of the ks
            if random.random() < 0.5 \
                    and not False in [x < y for x, y in
                                      zip([a+b for a,b in
                                           zip(accumulated_resources, list_items[j].weight)], ks.constraints)]:
                ks.ks[j] = 1
                # doesn't compute fitness as it is calculated via a method of ks
                # fitness = fit(ks)

                # Add all sizes of current item to the accumulated resources
                accumulated_resources = [x+y for x,y in zip(accumulated_resources, list_items[j].weight)]
        population += [ks]

    return population

#p = pop_init(10, items[0], knapsack[0])

17569
18015
17731
17122
18979
17909
17878
17693
18107
17744
0


### Selection

In [ ]:
def select_wolves(population):
    """
    Select the 3 best wolves

    :param population: Population in which we search
    :type population: list of Knapsack

    :rtype: (Knapsack, Knapsack, Knapsack)
    :return: The 3 best wolves
    """
    return sorted(population, key=lambda x: x.fit(), reverse=True)[-3:]



### generation of solution

##### $x^{p}_{j}(t) = \omega_{\alpha}*x^{\alpha}_{j}(t) + \omega_{\beta}*x^{\beta}_{j}(t) + \omega_{\delta}*x^{\delta}_{j}(t) + \epsilon(t)$

In [ ]:
# Generate an estimation of the position of the "prey" aka wanted solution
# Problem, the solution isn't binary
def generate_prey(alpha, beta, delta):
    """
    Generate the solution we search (the prey)

    :param alpha: The alpha wolf
    :type alpha: Knapsack
    :param beta: The beta wolf
    :type beta: Knapsack
    :param delta: The delta wolf
    :type delta: Knapsack

    :rtype: list of float
    :return: The "prey" as a list of float
    """
    total_fitness = alpha.fit() + beta.fit() + delta.fit()
    weightA = alpha.fit()/total_fitness
    weightB = beta.fit()/total_fitness
    weightD = delta.fit()/total_fitness

    return [alpha.ks[x]*weightA+beta.ks[x]*weightB+delta.ks[x]*weightD
            for x in range(len(alpha.ks))]

$y = x^{p}_{j}(t) - r1 * |x^{p}_{j}(t) - x^{k}_{j}(t)| $ </br>
$z^{k}_{j} = \begin{cases} 1, if \; \; r_{2} < \phi(y) \\ 0, otherwise \end{cases}$ </br>
$\phi(y)$ -> transform function -> 6 different see paper </br>
$\phi(y) = |tanh(y)|$ is the best ranked according to paper

In [ ]:
# Generate a trial solution
def generate_sol(prey, individual, transform_function=math.tanh):
    """
    Generate a trial solution from the prey

    :param prey: The researched solution
    :type prey: list of float
    :param individual: The current individual
    :type individual: Knapsack
    :param transform_function: Transform function real to integer
    :type transform_function: typing.Callable

    :rtype: Knapsack
    :return: A trial solution
    """
    r = random.uniform(-2, 2)
    trial_list = [x - r * abs(x - y) for x, y in zip(prey, individual.ks)]
    trial = [1 if random.uniform(0, 1) < transform_function(x) else 0 for x in trial_list]

    ks = Knapsack(individual.constraints, individual.items)
    ks.ks = trial

    return ks

### repair

In [ ]:
def repair(individual):
    pass

### Algorithm

In [ ]:
# Init
# generate sol + repair + fit
# inclusion

def gwo(max_iteration, list_items, list_constraints):
    pop = pop_init(max_iteration, list_items, list_constraints)
    alpha, beta, delta = select_wolves(pop)

    for iteration in range(max_iteration):
        # Estimate location of prey
        prey = generate_prey(alpha, beta, delta)

        for individual in pop():
            # Can remove the for loop with list comprehension
            #for dimension in individual.ks:
            trial_solution = generate_sol(prey, individual)

            trial_solution = repair(trial_solution)

            # Might want to make it differently
            # Or call fit(trial_solution) and have it make trial_solution.fit()
            if trial_solution.fit() > individual.fit():
                individual = trial_solution

                if trial_solution.fit() > alpha.fit():
                    alpha = trial_solution

                if trial_solution.fit() > beta.fit():
                    beta = trial_solution

                if trial_solution.fit() > delta.fit():
                    delta = trial_solution
            else:
                individual = trial_solution



